# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить два задания

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 5 баллов

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 5 баллов

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам.


__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [ ]:
# !pip install wandb
!pip3 install pytorch_lightning torchmetrics

In [ ]:
!pip install wandb

In [1]:
# Возможно ниже будет не очень работать WandbLogger с первого разу, перезапустите kernel тетрадки
import wandb

wandb.login()

wandb: Currently logged in as: krot. Use `wandb login --relogin` to force relogin


True

In [2]:
import math
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from pytorch_lightning.loggers import WandbLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy 
from torchvision.datasets import ImageFolder
from tqdm import tqdm

# You may add any imports you need

In [3]:
def seed_everything(seed):
    # Фискирует максимум сидов.
    # Это понадобится, чтобы сравнение оптимизаторов было корректным
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(123456)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
#!wget https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip?dl=1 -O dataset.zip && unzip -q dataset.zip

## Задание 0

### Что поможет сделать на 10 из 10 (одно задание - 5 баллов)

1. Использовать все возможные методы оптимизации и эксперемнтировать с ними.
2. Подбор learning rate. Пример из прошлого семинара как это делать: [Как найти lr](https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html)

```
  trainer = pl.Trainer(accelerator="gpu", max_epochs=2, auto_lr_find=True) 

  trainer.tune(module, train_dataloader, eval_dataloader)

  trainer.fit(module, train_dataloader, eval_dataloader))
```



3. Аугментация данных. [Документация (полезная)](https://pytorch.org/vision/main/transforms.html), а также [библиотека albumentation](https://towardsdatascience.com/getting-started-with-albumentation-winning-deep-learning-image-augmentation-technique-in-pytorch-47aaba0ee3f8)
4. Подбор архитектуры модели. 
5. Можно написать модель руками свою в YourNet, а можно импортировать не предобученную сетку известной архитектуры из модуля torchvision.models. Один из способов как можно сделать: 

  * `torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)`
  * Документация по возможным моделям и как их можно брать: [Документация (полезная)](https://pytorch.org/vision/stable/models.html)
6. Правильно нормализовывать данные при создании, пример [тык, но тут и в целом гайд от и до](https://www.pluralsight.com/guides/image-classification-with-pytorch)
7. Model Checkpointing. Сохраняйте свой прогресс (модели), чтобы когда что-то пойдет не так вы сможете начать с этого места или просто воспроизвести свои результаты модели, которые обучали. 
 * Пример как можно с wandb тут: [Сохраняем лучшие модели в wandb](https://docs.wandb.ai/guides/integrations/lightning)
 * По простому можно так: [Сохраняем модели в pytorch дока](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

### Подготовка данных

In [5]:
train_transform = transforms.Compose(
    [
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)
val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


In [6]:
# YOU CAN DEFINE AUGMENTATIONS HERE

#train_dataset = ImageFolder('/dataset/dataset/train', transform=train_transform)
#val_dataset = ImageFolder('/dataset/dataset/val', transform=val_transform)
train_dataset = ImageFolder('C:/Users/Крот/Documents/GitHub/Iad_git/sem_3_hw_2/dataset/dataset/train',transform=train_transform)
val_dataset = ImageFolder('C:/Users/Крот/Documents/GitHub/Iad_git/sem_3_hw_2/dataset/dataset/val', transform=val_transform)
# REPLACE ./dataset/dataset WITH THE FOLDER WHERE YOU DOWNLOADED AND UNZIPPED THE DATASET

train_dataloader = DataLoader(train_dataset, 
    batch_size=16,
    shuffle=True, 
    num_workers=1) # YOUR CODE HERE
val_dataloader = DataLoader(val_dataset, 
    batch_size=16,
    shuffle=False, 
    num_workers=1) # YOUR CODE HERE

In [7]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Посмотрим на картиночки

In [8]:
for batch in train_dataloader:
    images, class_nums = batch
    for i in range(len(images)):
        #plt.imshow(images[i].permute(1, 2, 0))
        print(images[i].size())
        #plt.show()
        #plt.imshow(images[10].permute(1, 2, 0))
        #plt.show()
    break

torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])


## Задание 1. 

5 баллов
Добейтесь accuracy на валидации не менее 0.44. В этом задании запрещено пользоваться предобученными моделями и ресайзом картинок.


Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались)
4. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

In [9]:
for i in range(2):
    print(i)

0
1


### Модель (или просто импортируйте не предобученную)

In [10]:
class YourNet(torch.nn.Module):
    def __init__(self,learning_rate,dropout=0.4):
        super().__init__()
        # -------------------------------------------
        # Архитектура сетки
        # -------------------------------------------
        self.batch_norm1 = torch.nn.BatchNorm2d(3)
        #self.batch_norm3 = torch.nn.BatchNorm1d(512)
        #self.batch_norm4 = torch.nn.BatchNorm1d(1024)
        self.conv0 = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ELU())
        self.conv1 = torch.nn.Sequential( 
            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding = 1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ELU(),
            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding = 1),
            torch.nn.Dropout(p=dropout),
            torch.nn.BatchNorm2d(64))
        self.conv2_0 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,stride = 2,padding=1)
        self.conv2_1 = torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding=1)
        self.conv2 = torch.nn.Sequential(
            torch.nn.BatchNorm2d(128),
            torch.nn.ELU(),
            torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding=1),
            torch.nn.Dropout(p=dropout),
            torch.nn.BatchNorm2d(128)
        )
        self.conv3_0 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,stride = 2,padding=1)
        self.conv3_1 = torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding=1)
        self.conv3 = torch.nn.Sequential(
            torch.nn.BatchNorm2d(256),
            torch.nn.ELU(),
            torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding=1),
            torch.nn.Dropout(p=dropout),
            torch.nn.BatchNorm2d(256)
        )
        self.conv4_0 = torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3,stride = 2,padding=1)
        self.conv4_1 = torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,padding=1)
        self.conv4 = torch.nn.Sequential(
            torch.nn.BatchNorm2d(512),
            torch.nn.ELU(),
            torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,padding=1),
            torch.nn.Dropout(p=dropout),
            torch.nn.BatchNorm2d(512)
        )
        self.linear1 = torch.nn.Sequential(torch.nn.Linear(in_features=512, out_features=1024),
         torch.nn.ELU(),
         torch.nn.BatchNorm1d(1024))                                  
        self.linear2 = torch.nn.Sequential(torch.nn.Linear(in_features=1024, out_features=4096),
         torch.nn.ReLU(),
         torch.nn.BatchNorm1d(4096)) 
        self.linear3 = torch.nn.Sequential(torch.nn.Linear(in_features=4096, out_features=1024),
         torch.nn.ReLU(),
         torch.nn.BatchNorm1d(1024))  
        self.output = torch.nn.Linear(in_features=1024, out_features=200)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AvgPool2d(kernel_size=4, stride=1)
        self.relu = torch.nn.ELU()
        self.concat1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=2)
        self.concat2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride=2)
        self.concat3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=2)
        # -------------------------------------------
        #Вспомогательные параметры и функции
        # -------------------------------------------
        self.loss_func = torch.nn.CrossEntropyLoss()
        self.targets = torch.Tensor()
        self.preds = torch.Tensor()

        
    def _forward(self, x):
      #block - 0
        x = self.conv0(x)
        y = self.maxpool(x)
      # block - 1
        x = self.conv1(y)
        y= self.relu(x+y)
        x = self.conv1(y)
        y= self.relu(x+y)
        x = self.conv1(y)
        y= self.relu(x+y)
      #block - 2
        x = self.conv2_0(y)
        x = self.conv2(x)
        y= self.relu(self.concat1(y)+x)
        x = self.conv2_1(y)
        x=self.conv2(x)
        y=self.relu(x+y)
        x = self.conv2_1(y)
        x=self.conv2(x)
        y=self.relu(x+y)
        x = self.conv2_1(y)
        x=self.conv2(x)
        y=self.relu(x+y)
      #block - 3
        x = self.conv3_0(y)
        x=self.conv3(x)
        y=self.relu(self.concat2(y)+x)
        x = self.conv3_1(y)
        x=self.conv3(x)
        y=self.relu(x+y)
        x = self.conv3_1(y)
        x=self.conv3(x)
        y=self.relu(x+y)
        x = self.conv3_1(y)
        x=self.conv3(x)
        y=self.relu(x+y)
        x = self.conv3_1(y)
        x=self.conv3(x)
        y=self.relu(x+y)
        x = self.conv3_1(y)
        x=self.conv3(x)
        y=self.relu(x+y)
      # block - 4
        x = self.conv4_0(y)
        x=self.conv4(x)
        y=self.relu(self.concat3(y)+x)
        x = self.conv4_1(y)
        x=self.conv4(x)
        y=self.relu(x+y)
        x = self.conv4_1(y)
        x=self.conv4(x)
        y=self.relu(x+y)
        x = self.avgpool(y)
      # block - classifier
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        return self.output(x)
    
    def forward(self, images, target=None):
        output = self._forward(images)
        # get accuracy score and save it to self.accuracy
        if target is not None:
            loss = self.loss_func(output, target)

            self.targets = torch.cat((self.targets, target.cpu()), 0)
            pred = torch.argmax(output, dim=-1)
            self.preds = torch.cat((self.preds, pred.cpu()), 0)
            self.accuracy = accuracy(self.preds.to(torch.int64), self.targets.to(torch.int64))

        return loss
    
        
    def get_accuracy(self, reset=False):
        # return accuracy by all values till now
        if reset:
            self.targets = torch.Tensor()
            self.preds = torch.Tensor() 
        return self.accuracy

### Тренировочный класс lightning

In [11]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.model = model
        self.lr = learning_rate
        pass

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        images, target = train_batch
        loss = self.model(images, target)
        self.log("train_loss", loss, prog_bar=True)  # сохраняет логи в папку, но можно несложно добавить wandb
        self.log("accuracy_train",model.get_accuracy(),prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        images, target = val_batch
        loss = self.model(images, target)
        self.log("val_loss", loss, prog_bar=True)
        self.log("accuracy_val",model.get_accuracy(),prog_bar=True)

In [12]:
lr = 1e-3

In [13]:
wandb_logger = WandbLogger(log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning

model = YourNet(lr).to(device) # YOUR CODE HERE
module = YourModule(
model,
learning_rate=lr ) # YOUR CODE HERE

In [14]:
# trainer = pl.Trainer(logger=wandb_logger)  # YOUR CODE HERE
# trainer = pl.Trainer(
#     accelerator="cpu",
#     max_epochs=2
# )
# trainer.fit(module, train_dataloader, val_dataloader)
# 3.5) we can also find best learning rate like this: https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html
trainer = pl.Trainer(logger=wandb_logger,accelerator="cpu", max_epochs=2, auto_lr_find=True)    
trainer.tune(module, train_dataloader, val_dataloader)
trainer.fit(module, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=97` reached.
LR finder stopped early after 97 steps due to diverging loss.
Learning rate set to 0.0004365158322401656
Restoring states from the checkpoint path at C:\Users\Крот\Documents\GitHub\Iad_git\sem_3_hw_2\.lr_find_1d8d73db-cfde-4ed7-aabd-557e6ea42754.ckpt

  | Name  | Type    | Params
----------------------------------
0 | model | YourNet | 17.1 M
----------------------------------
17.1 M    Trainable params
0         Non-trainable params
17.1 M    Total params
68.532    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### Валидация результатов задания

In [ ]:
def evaluate_task(model, test_dataloader):
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)
    model.eval()
    accuracy = 0.0
    for images, labels in tqdm(test_dataloader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            loss = model(images, labels)
            acc_batch = model.get_accuracy(reset=False)
        accuracy += acc_batch
    accuracy = accuracy / len(test_dataloader)
    return accuracy

In [ ]:
model = model

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * accuracy / 0.44, 0, 10):.2f} баллов")

In [ ]:
print(accuracy)

In [ ]:
model.get_accuracy(reset=True)

## Задание 2

5 баллов
Добейтесь accuracy на валидации не менее 0.84. В этом задании делать ресайз и использовать претрейн можно.

Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались или можете пойти дальше).
4. Попробуйте сначала посмотреть качество исходной модели без дообучения, сохраните как baseline. Отсюда поймете какие слои нужно дообучать.
5. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

### Модель (или просто импортируйте предобученную)

In [ ]:
class YourNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        pass

    def _forward(self, x):
        # runs the Neural Network
        # YOUR CODE HERE
        pass

    def forward(self, images, target=None):
        # YOUR CODE HERE
        pass

    def get_accuracy(self, reset=False):
        # YOUR CODE HERE
        pass

### Тренировочный класс lightning

In [ ]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        # YOUR CODE HERE
        pass

    def forward(self, x):
        # YOUR CODE HERE
        pass

    def configure_optimizers(self):
        # YOUR CODE HERE
        pass

    def training_step(self, train_batch, batch_idx):
        # YOUR CODE HERE
        pass

    def validation_step(self, val_batch, batch_idx):
        # YOUR CODE HERE
        pass

In [ ]:
wandb_logger = WandbLogger(log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning
device =  # YOUR CODE HERE

model = YourNet().to() # YOUR CODE HERE
module = YourModule() # YOUR CODE HERE

In [ ]:
trainer = pl.Trainer(logger=wandb_logger)  # YOUR CODE HERE
trainer.fit()  # YOUR CODE HERE

### Валидация результатов задания

In [ ]:
model = # Подгрузить свою сохраненную модель сюда

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * (accuracy - 0.5) / 0.34, 0, 10):.2f} баллов")

# Отчёт об экспериментах 

текст писать тут (или ссылочку на wandb/любой трекер экспреиментов) для каждого задания, то есть не обязательно именно тут рисовать графики, если вы используете готовые трекеры/мониторинги ваших моделей.